In [1]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
from itertools import combinations
import networkx as nx
import pandas as pd
import itertools
import copy

In [2]:
from math import radians, cos, sin, asin, sqrt
AVG_EARTH_RADIUS = 6372.8

def haversine(point1, point2):

    # извлекаем долготу и широту
    lat1, lng1 = point1
    lat2, lng2 = point2

    # переводим все эти значения в радианы
    lat1, lng1, lat2, lng2 = map(radians, (lat1, lng1, lat2, lng2))

    # вычисляем расстояние по формуле
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = sin(lat * 0.5) ** 2 + cos(lat1) * cos(lat2) * sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * asin(sqrt(d))
    return h 

import numpy as np
def haversine_np(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km =  6372.8 * c
    return km


In [3]:
file_id = '5'

In [4]:
points = pd.read_csv(f'points{file_id}.txt', header=None, names=["raw_data"])
points[['Latitude', 'raw_data']]= points["raw_data"].str.split(n = 1, expand = True)
points[['Longitude', 'raw_data']]= points["raw_data"].str.split(n = 1, expand = True)
points.rename(columns={'raw_data': 'city'}, inplace=True)
points = points.astype({'Latitude': 'float', 
                        'Longitude': 'float'})

# # Делает город индексом #
# points.index=points.city
# del points["city"]
# points.index.name = None

# points_dict := {city: (Latitude, Longitude)}
points_dict = points.set_index('city').T.to_dict('list')

In [5]:
graph = pd.read_csv(f'graph{file_id}.txt', header=None, index_col=None, names=["raw_data"])
graph[['node1', 'node2']] = graph["raw_data"].str.split(" --- ", n = 1, expand = True)
del graph['raw_data']
# graph["color"] = "red"
# graph["estimate"] = 1
# graph["trail"] = 0.18
# graph["distance"] = 10

# добавляю дистанцию
graph = pd.merge(graph, points, right_on="city", left_on="node1")
del graph['city']
graph = pd.merge(graph, points, right_on="city", left_on="node2", suffixes=('_node1', "_node2"))
del graph['city']
graph['weight'] = haversine_np(
    graph['Latitude_node1'],  graph['Longitude_node1'], 
    graph['Latitude_node2'],  graph['Longitude_node2']
)
del graph['Latitude_node1'],  graph['Longitude_node1'], 
del graph['Latitude_node2'],  graph['Longitude_node2']


Создаю граф для поиска в нём не связанных мини-графов

In [6]:
G=nx.Graph()
G.add_nodes_from(points.city.to_list())
weighted_edges_from = graph.values

G.add_weighted_edges_from(weighted_edges_from)

Нахожу не связанные мини-графы

In [7]:
%%time
from collections import defaultdict
def connected_components_list(G):
    viwes = []
    components = nx.connected_components(G)
    result = []
    for nodes in components:
        for node in nodes:
            if node not in viwes:
                result.append(tuple(nodes))
                viwes.extend(nodes)
    return result


connected_components = connected_components_list(G)
print("Найдено связанных графов:", len(connected_components))

Найдено связанных графов: 1630
Wall time: 632 ms


In [8]:
connected_components_index_dict = {} # {int[index]: List[mini-graph]}
for i in range(len(connected_components)):
    connected_components_index_dict[i] = connected_components[i]

Создаю граф, в котором есть все связи межу мини-графами,
* вершины: названия мини-графов
* рёбра: дистанция

In [9]:
%%time
new_graph = []
viwes_graph_idx = []
C = 0
# 18 484 363 итераций 
for graph_index1, graph_index2 in combinations(connected_components_index_dict.keys(), 2):
    min_weight = "None"
    graph1, graph2 = connected_components_index_dict[graph_index1], connected_components_index_dict[graph_index2]
    for node1 in  graph1:
        for node2 in  graph2:
            if min_weight == "None":
                min_weight = haversine(points_dict[node1], points_dict[node2])
            else:
                min_weight = min(min_weight, haversine(points_dict[node1], points_dict[node2]))
    new_graph.append([graph_index1, graph_index2, min_weight])

Wall time: 36.1 s


In [10]:
fuck_graph = pd.DataFrame(new_graph)
fuck_graph.rename(
    columns={0: 'node1',
             1: "node2",
             2: "weight"},
             inplace=True)
fuck_graph = fuck_graph.astype({
    'node1': 'str', 
    'node2': 'str'})

In [11]:
Fucking_G=nx.Graph()
Fucking_G.add_nodes_from(list(map(str, connected_components_index_dict.keys())))
weighted_edges_from = fuck_graph.values

Fucking_G.add_weighted_edges_from(weighted_edges_from)

Нахожу minimum spanning tree от подученного графа

In [12]:
fucking_mst = nx.minimum_spanning_tree(Fucking_G)
result_sum = 0
for u,v,d in fucking_mst.edges(data=True):
    result_sum+=d['weight']
print('Итог:', result_sum)

Итог: 170775.80728912327
